In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/upcoming-event.csv')
len(df)

4421

In [6]:
num_upcoming_fights = len(df_upcoming)

In [7]:
df = df_upcoming.append(df)
len(df)

4432

In [8]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [10]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [11]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

11
11
11
4421
4421
4421


In [12]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\Matt-Predator\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Marlon Moraes,Cory Sandhagen,115,-141,115.000000,70.921986,2020-10-10,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,Edson Barboza,Makwan Amirkhani,-286,225,34.965035,225.000000,2020-10-10,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Ben Rothwell,Marcin Tybura,-159,130,62.893082,130.000000,2020-10-10,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,Markus Perez,Dricus Du Plessis,133,-167,133.000000,59.880240,2020-10-10,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,Tom Aspinall,Alan Baudot,-560,400,17.857143,400.000000,2020-10-10,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4416,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4417,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4418,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4419,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Brian Ortega,Chan Sung Jung,160,-200,160.000000,50.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Katlyn Chookagian,Jessica Andrade,130,-159,130.000000,62.893082,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jimmy Crute,Modestas Bukauskas,-335,250,29.850746,250.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Claudio Silva,James Krause,120,-150,120.000000,66.666667,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thomas Almeida,Jonathan Martinez,-139,110,71.942446,110.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Mateusz Gamrot,Guram Kutateladze,-335,245,29.850746,245.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Gillian Robertson,Poliana Botelho,-215,169,46.511628,169.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Junyong Park,John Phillips,-278,215,35.971223,215.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Jamie Mullarkey,Fares Ziam,-137,110,72.992701,110.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Gadzhimurad Antigulov,Maxim Grishin,277,-375,277.000000,26.666667,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [15]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]

    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    

    return(0)

In [16]:
with open('../automated_model_creation/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice:
model_choice = 10

print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

[['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20', 'model_9_2_20', 'model_9_10_20'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=577, random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)", 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)', 'GaussianNB(var_smoothing=1e-08)', 'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'GaussianNB(var_smoothing=1e-12)', 'GaussianNB(var_smoothing=1e-12)'], ['["R_Women\'s Featherweight_rank", \'loss_dif\', \'R_avg_SUB_ATT\', \'lose_streak_dif\', \'B_avg_SIG_STR_lande

In [17]:

get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Brian Ortega vs. Chan Sung Jung
Brian Ortega has an EV of 38.62 on odds of 160.  They have 53.32 % chance of winning.

Katlyn Chookagian vs. Jessica Andrade
No bets.  EV1:-13.03956656683939. EV2: 1.3050252688302564
Katlyn Chookagian has an EV of -13.04 on odds of 130.  They have 37.81 % chance of winning.

Jimmy Crute vs. Modestas Bukauskas
Jimmy Crute has an EV of 28.5 on odds of -335.  They have 98.96 % chance of winning.

Claudio Silva vs. James Krause
Claudio Silva has an EV of 109.65 on odds of 120.  They have 95.29 % chance of winning.

Thomas Almeida vs. Jonathan Martinez
Thomas Almeida has an EV of 12.92 on odds of -139.  They have 65.67 % chance of winning.

Mateusz Gamrot vs. Guram Kutateladze
Guram Kutateladze has an EV of 46.56 on odds of 245.  They have 42.48 % chance of winning.

Gillian Robertson vs. Poliana Botelho
Gillian Robertson has an EV of 24.72 on odds of -215.  They have 85.12 % chance of winning.

Junyong Park vs. John Phillips
Junyong Park has an EV of 15.92 o

0